In [11]:
!pip install python-telegram-bot
!pip install git+https://github.com/python-telegram-bot/python-telegram-bot@v13.8
!pip install PyPDF2
!pip install python-telegram-bot[job-queue]
!pip install transformers



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/python-telegram-bot/python-telegram-bot (to revision v13.8) to /tmp/pip-req-build-5yg1yctz
  Running command git clone --filter=blob:none --quiet https://github.com/python-telegram-bot/python-telegram-bot /tmp/pip-req-build-5yg1yctz
  Running command git checkout -q 51a4a6664cbf7663e56d1be8ba03bd2d22863783
  Resolved https://github.com/python-telegram-bot/python-telegram-bot to commit 51a4a6664cbf7663e56d1be8ba03bd2d22863783
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, http

In [16]:
import os
import tempfile
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler
from PyPDF2 import PdfReader
from transformers import pipeline

# Telegram bot token
TOKEN = 'YOUR_BOT_TOKEN'

# Initialize the question answering pipeline
qa_pipeline = pipeline('question-answering')

def start(update: Update, context):
    """Handler for the /start command."""
    update.message.reply_text('Hello! Send me a PDF document to learn from.')

def learn_from_pdf(update: Update, context):
    """Handler for PDF documents."""
    file = context.bot.get_file(update.message.document.file_id)
    with tempfile.NamedTemporaryFile(suffix='.pdf') as temp_file:
        file.download(temp_file.name)

        # Read the PDF content
        with open(temp_file.name, 'rb') as pdf_file:
            reader = PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()

        # Perform learning operations with the extracted text

        # Send a response to the user
        update.message.reply_text('PDF content has been processed and learned.')

def answer_question(update: Update, context):
    """Handler for answering questions."""
    question = update.message.text
    answer = qa_pipeline({
        'question': question,
        'context': context.user_data['text']
    })

    # Send the answer to the user
    update.message.reply_text(answer['answer'])

def main():
    """Main function to start the Telegram bot."""
    updater = Updater(TOKEN, use_context=True)
    dispatcher = updater.dispatcher

    # Handlers
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(MessageHandler(None, learn_from_pdf))
    dispatcher.add_handler(MessageHandler(None, answer_question))

    # Start the bot
    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()


No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


TypeError: ignored